# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<b>Инструкция по выполнению проекта</b>

1.Загрузите и подготовьте данные. Поясните порядок действий.

2.Обучите и проверьте модель для каждого региона:

    2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
    2.2. Обучите модель и сделайте предсказания на валидационной выборке.
    2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
    2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
    2.5. Проанализируйте результаты.

3.Подготовьтесь к расчёту прибыли:

    3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
    3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
    3.3. Напишите выводы по этапу подготовки расчёта прибыли.

4.Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

    4.1. Выберите скважины с максимальными значениями предсказаний.
    4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
    4.3. Рассчитайте прибыль для полученного объёма сырья.

5.Посчитайте риски и прибыль для каждого региона:

    5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
    5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
    5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

<b>Описание данных</b>

Данные геологоразведки трёх регионов находятся в файлах:

/datasets/geo_data_0.csv.

/datasets/geo_data_1.csv.

/datasets/geo_data_2.csv.
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

Импорт необходимых инструментов

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Загрузка данных и ознакомление с ними

In [ ]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

display(data_0.head())
display(data_0.info())

display(data_1.head())
display(data_1.info())

display(data_2.head())
data_2.info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Проверка датафреймвов на наличие явных дубликатов

In [ ]:
display(data_0.duplicated().sum())
display(data_1.duplicated().sum())
data_2.duplicated().sum()

0

0

0

### Вывод

- Пропусков в данных не обнаружено
- Размер трёх выборок одинаковый
- Дубликатов в данных не обнаружено
- Данные представлены в одном мастштабе
- Все признаки являются численными

## Обучение и проверка модели

### Работа с данными первого региона

#### Деление данных

Деление данных первого региона, используя функцию train_test_split на две выборки: обучающую (75%) и валидационную (25%).

Вывод результатов на экран.

In [ ]:
df_0_train, df_0_valid = train_test_split(data_0, test_size=.25, random_state=12345)
display(df_0_train.shape)
df_0_valid.shape

(75000, 5)

(25000, 5)

Запишем в обучающие признаки данные из обучающей выборки, исключив ненужные столбцы. В целевой признак внесём столбец *product*. Такую же процедуру проделаем с валидационной выборкой.

In [ ]:
features_train_0 = df_0_train.drop(['id', 'product'], axis=1)
target_train_0 = df_0_train['product']
features_valid_0 = df_0_valid.drop(['id', 'product'], axis=1)
target_valid_0 = df_0_valid['product']

#### Обучение модели

Создание модели логистической регрессии, её обучение и сохранение предсказаний

In [ ]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_valid_0 = model_0.predict(features_valid_0)

#### Вычисления корня из среднеквадратичной ошибки (RMSE)

In [ ]:
rmse_0 = mean_squared_error(target_valid_0, predictions_valid_0) ** .5
rmse_0

37.5794217150813

#### Вычисление среднего запаса предсказанного сырья

In [ ]:
product_pred_mean_0 = predictions_valid_0.mean()
product_pred_mean_0

92.59256778438035

#### Вывод

- средний запас предсказанного сырья - 92.6
- отклонение 37.6

### Работа с данными второго региона

#### Деление данных

Деление данных второго региона, используя функцию train_test_split на две выборки: обучающую (75%) и валидационную (25%).

Вывод результатов на экран.

In [ ]:
df_1_train, df_1_valid = train_test_split(data_1, test_size=.25, random_state=12345)
display(df_1_train.shape)
df_1_valid.shape

(75000, 5)

(25000, 5)

Запишем в обучающие признаки данные из обучающей выборки, исключив ненужные столбцы. В целевой признак внесём столбец product. Такую же процедуру проделаем с валидационной выборкой.

In [ ]:
features_train_1 = df_1_train.drop(['id', 'product'], axis=1)
target_train_1 = df_1_train['product']
features_valid_1 = df_1_valid.drop(['id', 'product'], axis=1)
target_valid_1 = df_1_valid['product']

#### Обучение модели

Создание модели логистической регрессии, её обучение и сохранение предсказаний

In [ ]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)

#### Вычисления корня из среднеквадратичной ошибки (RMSE)

In [ ]:
rmse_1 = mean_squared_error(target_valid_1, predictions_valid_1) ** .5
rmse_1

0.893099286775617

#### Вычисление среднего запаса предсказанного сырья

In [ ]:
product_pred_mean_1 = predictions_valid_1.mean()
product_pred_mean_1

68.728546895446

#### Вывод

- средний запас предсказанного сырья 68.7
- отклонение - 0.89

### Работа с данными третьего региона

#### Деление данных

Деление данных третьего региона, используя функцию train_test_split на две выборки: обучающую (75%) и валидационную (25%).

Вывод результатов на экран.

In [ ]:
df_2_train, df_2_valid = train_test_split(data_2, test_size=.25, random_state=12345)
display(df_2_train.shape)
df_2_valid.shape

(75000, 5)

(25000, 5)

Запишем в обучающие признаки данные из обучающей выборки, исключив ненужные столбцы. В целевой признак внесём столбец product. Такую же процедуру проделаем с валидационной выборкой.

In [ ]:
features_train_2 = df_2_train.drop(['id', 'product'], axis=1)
target_train_2 = df_2_train['product']
features_valid_2 = df_2_valid.drop(['id', 'product'], axis=1)
target_valid_2 = df_2_valid['product']

#### Обучение модели

Создание модели логистической регрессии, её обучение и сохранение предсказаний

In [ ]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)

#### Вычисления корня из среднеквадратичной ошибки (RMSE)

In [ ]:
rmse_2 = mean_squared_error(target_valid_2, predictions_valid_2) ** .5
rmse_2

40.02970873393434

#### Вычисление среднего запаса предсказанного сырья

In [ ]:
product_pred_mean_2 = predictions_valid_2.mean()
product_pred_mean_2

94.96504596800489

#### Вывод

- средний запас предсказанного сырья - 95
- отклонение - 40

### Вывод

- Для каждого региона обучена модель логистической регрессии и получены предсказания среднего запаса сырья в скважине
- Предсказанный средний запас сырья в скважине первого региона 92.6 при отклонении 37.6
- Предсказанный средний запас сырья в скважине второго региона 68.7 при отклонении 0.89
- Предсказанный средний запас сырья в скважине третьего региона 95 при отконении 40
- Наименьшее отклонение во втором регионе, но в нём в среднем меньше запасов сырья, следовательно, потребуются дополнительлные исследования

## Подготовка к расчёту прибыли

<b>Сохранение ключевых значений в отдельных переменных</b>

Преобразование целевых признаков и предсказаний моделей в объект *Series*

In [ ]:
target_valid_0 = pd.Series(target_valid_0).reset_index(drop=True)
predictions_valid_0 = pd.Series(predictions_valid_0).reset_index(drop=True)

target_valid_1 = pd.Series(target_valid_1).reset_index(drop=True)
predictions_valid_1 = pd.Series(predictions_valid_1).reset_index(drop=True)

target_valid_2 = pd.Series(target_valid_2).reset_index(drop=True)
predictions_valid_2 = pd.Series(predictions_valid_2).reset_index(drop=True)

Предоставленные данные

In [ ]:
UNIT_INCOME = 450000
TOTAL_COUNT = 500
COUNT = 200
BUDGET = 10000000000
SAMPLES_NUMBER = 1000

<b>Достаточный объём сырья для безубыточной разработки новой скважины</b>

Деление заложенного буджета на стоимоcть единицы продукта и на количество разрабатываемых скважин

In [ ]:
enough_raw_oil = BUDGET / UNIT_INCOME /COUNT
display(enough_raw_oil)

111.11111111111111

### Вывод

Средний запас сырья скважин в каждом разрабатываемом регионе ниже необходимого для безубыточной разработки запаса:

- Необходимый запас - 111

## Расчёт прибыли и рисков 

### Создание функции для расчёта прибыли по выбранным скважинам и предсказаниям модели

- На вход функция принимает целевые значения объёмов запасов скважин, предсказания модели по запасам и количество точек для разработки.
- В теле функции: выбор скважины с максимальными значениями предсказаний, суммирование целевых значений объёмов сырья, соответствующее этим предсказаниям.
- На выходе: прибыль для полученного объёма сырья.

In [ ]:
def revenue(target, predictions, COUNT):
    preds_sorted = predictions.sort_values(ascending=False)
    selected = target[preds_sorted.index][:COUNT]
    return UNIT_INCOME * selected.sum() - BUDGET

Проверка работоспособности функции

In [ ]:
revenue(target_valid_0, predictions_valid_0, COUNT)

3320826043.1398506

### Расчёт рисков и прибыли для каждого региона

Создание цикла, который расчитает прибыль и риски для регионов

In [ ]:
target = [target_valid_0, target_valid_1, target_valid_2]
predictions = [predictions_valid_0, predictions_valid_1, predictions_valid_2]
regions = ['Первый регион', 'Второй регион', 'Третий регион']

state = np.random.RandomState(12345)

for m in range(len(target)):
    values = []
    for i in range(SAMPLES_NUMBER):
        target_subsample = target[m].sample(n=TOTAL_COUNT, replace=True, random_state=state)
        predictions_subsample = predictions[m].loc[target_subsample.index]
        values.append(revenue(target_subsample.reset_index(drop=True), predictions_subsample.reset_index(drop=True), COUNT))

    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)

    mean = values.mean()
    
    loss_probability = sum(values < 0) / values.count() * 100
    
    print(regions[m])
    print("Средняя возможная прибыль:", mean)
    print("95%-доверительный интервал:", "от", lower, "до", upper)
    print("Вероятность убытков:", loss_probability, "%")
    

Первый регион
Средняя возможная прибыль: 396164984.8023711
95%-доверительный интервал: от -111215545.89049526 до 909766941.5534226
Вероятность убытков: 6.9 %
Второй регион
Средняя возможная прибыль: 461155817.2772397
95%-доверительный интервал: от 78050810.7517417 до 862952060.2637234
Вероятность убытков: 0.7000000000000001 %
Третий регион
Средняя возможная прибыль: 392950475.17060447
95%-доверительный интервал: от -112227625.37857565 до 934562914.5511636
Вероятность убытков: 6.5 %


### Вывод

- Чтобы найти распределение прибыли, применили технику Bootstrap с 1000 выборок
- Вероятность убытков во втором регионе минимальная и возможная средняя прибыль у него самая высокая

## Общий вывод

- Для выполнения поставленной задачи были обработаны данные трёх регионов с 10000 месторождений в каждом.
- Для составления прогноза по объёму запасов в каждом регионе была создана модель линейной регрессии.
- Наиболее точные предсказания оказались у модели второго региона: средний запас сырья в скважине 68.7 при отклонении 0.89, предсказания по первому и третьему региону получились схожими между собой
- Рассчитан достаточный объём сырья для безубыточной разработки новой скважины - 111
- Для расчёта распределения прибыли применена техника *Bootstrap* с 1000 выборок, которая показала, что <b>наиболее подходящий для разработки регион - второй</b>, так как у него наименьшая вероятность убытков и наибольшая средняя возможная прибыль